In [ ]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('xtrain_obfuscated.txt',header=None)
df_Y_train = pd.read_csv('ytrain.txt',header=None)

df_Y_train.rename(columns={0:'y_train'},inplace=True)
df_train.rename(columns={0:'x_train'},inplace=True)

train = pd.concat([df_train,df_Y_train],axis=1)

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
train.isnull().any()

x_train    False
y_train    False
dtype: bool

In [ ]:
train.shape

(32513, 2)

In [ ]:
train['x_train'][32512]

'twiwtvenvipmmvulqvtwkrszuhqvmvamuhsktwmktwmvamypenuhskenamuhtwiwqvgzpmokuhraleqjuhtwletwamdfuhtwleulentwamuluhtwamguuhrasaeniwgzuhtwamuluhpmuluhpmlruhqvskleendfletwulxeenuhskiwkrpmdfuhiguhulamlrkrpmamuluhiguhqgtwiwgzeztwguuhtwamuluhpmuluhtwezpmamuhulvitwdfuhiguhxemvuhtwezuhtwlepmxetwtvuhtwskkrulmviwpmqvuhskvienuhvipmmvulyptwbrtwskuhxepmuhqvlekrpmamuhskenamuhiguhqvenuhskvienuhpktwypvipmuhulenuhletwvivimvskuhamulmvdfuhxeiwtwqvsaez'

In [ ]:
corpus_lm = []
for i in range(0,len(train)):
    k = train['x_train'][i]
    #print(i)
    d1 = list(k)
    listToStr1 = ' '.join([str(elem) for elem in d1])
    corpus_lm.append(listToStr1)
           

In [ ]:
len(corpus_lm)

32513

In [ ]:
train['y_train'].value_counts()

7     5097
6     4226
3     4023
8     3634
1     3459
10    3052
4     2337
5     2283
2     1471
11    1408
9      980
0      543
Name: y_train, dtype: int64

In [ ]:
from tensorflow.keras.optimizers import Adam
size_voc = 26

onehot_repr=[one_hot(words,size_voc)for words in corpus_lm] 
onehot_repr[0]

sent_length=500
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

from tensorflow.keras.layers import Dropout
from keras.layers import LSTM,Bidirectional
## Creating model
embedding_vector_features=400
model=Sequential()
model.add(Embedding(size_voc,embedding_vector_features,input_length=sent_length))


model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(12,activation= 'softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


print(model.summary())

[[ 0  0  0 ... 19 16  3]
 [ 0  0  0 ... 19 16  3]
 [ 0  0  0 ... 19 16  3]
 ...
 [ 0  0  0 ... 17  3  8]
 [ 0  0  0 ... 23  1  9]
 [ 0  0  0 ... 20  1  6]]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 400)          10400     
                                                                 
 lstm (LSTM)                 (None, 100)               200400    
                                                                 
 dense (Dense)               (None, 1024)              103424    
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dropout_1 (Dropout)         (No

In [ ]:
import numpy as np
X_final_lstm = np.array(embedded_docs)
print(X_final_lstm.shape)

y = train['y_train']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final_lstm, y, test_size=0.3, random_state=42)

(32513, 500)


In [ ]:
history = model.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),batch_size=500)

Epoch 1/50
46/46 [==============================] - 156s 3s/step - loss: 2.3685 - accuracy: 0.1492 - val_loss: 2.3256 - val_accuracy: 0.1692
Epoch 2/50
46/46 [==============================] - 136s 3s/step - loss: 2.2964 - accuracy: 0.1748 - val_loss: 2.2743 - val_accuracy: 0.1916
Epoch 3/50
46/46 [==============================] - 139s 3s/step - loss: 2.2627 - accuracy: 0.1906 - val_loss: 2.2457 - val_accuracy: 0.1965
Epoch 4/50
46/46 [==============================] - 136s 3s/step - loss: 2.2461 - accuracy: 0.1941 - val_loss: 2.2235 - val_accuracy: 0.2000
Epoch 5/50
46/46 [==============================] - 134s 3s/step - loss: 2.2288 - accuracy: 0.1999 - val_loss: 2.2067 - val_accuracy: 0.1992
Epoch 6/50
46/46 [==============================] - 135s 3s/step - loss: 2.2090 - accuracy: 0.2026 - val_loss: 2.1923 - val_accuracy: 0.2010
Epoch 7/50
46/46 [==============================] - 135s 3s/step - loss: 2.2111 - accuracy: 0.2047 - val_loss: 2.2046 - val_accuracy: 0.1999
Epoch 8/50
46

In [ ]:
model.save('lstm_model.h5')

In [ ]:
y_pred= model.predict(X_test)

In [ ]:
y_pred

array([[0.01593952, 0.11001299, 0.05003873, ..., 0.03359369, 0.08939397,
        0.04864797],
       [0.00954076, 0.04109536, 0.11247541, ..., 0.04526624, 0.14895074,
        0.03981059],
       [0.01974044, 0.06984717, 0.05572156, ..., 0.03973183, 0.12478016,
        0.06861694],
       ...,
       [0.0094267 , 0.09505544, 0.05473486, ..., 0.03218085, 0.09911944,
        0.03632521],
       [0.01813178, 0.1396071 , 0.03137454, ..., 0.0187068 , 0.05968252,
        0.07244197],
       [0.02008087, 0.12135644, 0.05249487, ..., 0.03526597, 0.08410072,
        0.0503019 ]], dtype=float32)

In [ ]:
 
classes_x=np.argmax(y_pred,axis=1)

In [ ]:
classes_x

array([7, 3, 6, ..., 7, 3, 7])

In [ ]:
y_test

4421      7
821      10
10556    10
18535     6
16768     1
         ..
5116      6
27609    10
26257     4
8648      2
10718     1
Name: y_train, Length: 9754, dtype: int64

In [ ]:
from sklearn.metrics import accuracy_score
print("accuracy_score:", accuracy_score(y_test,classes_x))   ############# accuracy

accuracy_score: 0.22647119130613083


In [ ]:
df_test = pd.read_csv('xtest_obfuscated.txt',header=None)



In [ ]:
df_test[0]

0       tvletwgzkrqvuhtwamuluhpkskpmpmiwtvuhamqvmviwlr...
1       qvmvuhskleenmvviengzxyuhqvmvamguuhrakrpmsauhul...
2       twamuhulenamuluhskvienuhqgdftwmvmkletwulvimvuh...
3       sooatwqvvitwqvvipmviuhoaampmpmmwuhuhraulvikrpm...
4       lepmuhpmdfuluhtwletwdfuhtwletwamguuhuhraezpmpm...
                              ...                        
2995    enuhqvenuhqgeztwtwulqvtwuhtwiwtvenletwskmvqvvi...
2996    qvtwmkpmezuhulqvkrskuhtwamuluhulenamuluhskvien...
2997    ezmvamuhtwskenezuhulenamuluhulmvuhqvendfuhulen...
2998    skvikrpmtvuhskleendfulkrpmuhtwamuluhtwletwamdf...
2999    twamuluhiwiwmvuluhletwamultwlrpmuluhsktwgzezen...
Name: 0, Length: 3000, dtype: object

In [ ]:
input_test_data = np.array(df_test[0])

In [ ]:
type(input_test_data)

numpy.ndarray

In [ ]:
input_test_data

array(['tvletwgzkrqvuhtwamuluhpkskpmpmiwtvuhamqvmviwlrvikquhtwamuluhqgvipmmvulkriwpmqvtwleuhamqvmviwlrvikquhtwamuluhqgqvqvtwviezlemvxeuhamqvmviwlrvikquhtwamuluhpkskvieniwlrvikquhqvmvuhqgpmpmiwletwulenokuhxepmuhtwiwululentvuhtwamuluhvimvuhsktwlemvezskenuhtwtvuhulqvkrezuhamypmvamdfuhulenamguuhraskvipmyptwqvuhtwamuluhxepmuhvimvenulgzenypuhenuhsatvuhvipmdfuhqgletwsklepmuhulqvlemvxeuhtwamuluhxepmuhtwiwululentvuhenuhqvmvuhpmpmiwletwulenok',
       'qvmvuhskleenmvviengzxyuhqvmvamguuhrakrpmsauhulyptwulpmlegzuhskigbhbhkrpmsauhulyptwulpmlegzuhskigbhbhpmfquhraskiwlepmdfuhtwamuluhiwiwenuhlepmxeuhskentwamuhlekrpmsauhxepmuhlemvenamuhenuhullekramuhulviskiwkrpmdfuhiguhratwezuhxepmuhskmvenlexeenuhtwtvuhulvipmskuhqgsapmtvuhsaqjuhraletwskvikriwtvuhletwulxeenuhletwskvikriwtvuhtwskenezuhtwamuhskvienuhezmvamuhvipmgzkruhqvendfuhtwsatwuhqvvienezuhskiwpmuhtwamuluhulkrtv',
       'twamuhulenamuluhskvienuhqgdftwmvmkletwulvimvuhvienuhqvtwypvitwgzcimvqvuhvitwtwulxemvxeuhulenuhgzentwamypuhtwypvienulvimvenkrszypenuh

In [ ]:
corpus_lm2 = []
for i in range(0,len(df_test)):
    k = df_test[0][i]
    #print(i)
    d1 = list(k)
    listToStr1 = ' '.join([str(elem) for elem in d1])
    corpus_lm2.append(listToStr1)

In [ ]:
onehot_repr=[one_hot(words,size_voc)for words in corpus_lm2] 
onehot_repr[0]

sent_length=500
embedded_docs2=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs2)

[[ 0  0  0 ...  9 15 19]
 [ 0  0  0 ... 23  7 24]
 [ 0  0  0 ...  1  6 25]
 ...
 [ 0  0  0 ... 23  7 24]
 [ 0  0  0 ... 19 16  3]
 [ 0  0  0 ... 19  3 16]]


In [ ]:
############### test data 

type(embedded_docs2)

numpy.ndarray

In [ ]:
input_test = model.predict(embedded_docs2)

In [ ]:
classes_Input_test =np.argmax(input_test,axis=1)

In [ ]:
print(classes_Input_test)

[7 7 8 ... 7 7 6]


In [ ]:
### classes_Input_test ------------- output of xtest_obfuscated.txt 

len(classes_Input_test)

3000

In [ ]:


with open("ytest.txt", "w") as output:
    for row in classes_Input_test:
        output.write(str(row) + '\n')

In [ ]:
k ={'a': 11, 'c': 23, 'b': 26, 'e': 5, 'd': 19, 'g': 17, 'f': 20, 'i': 9, 'h': 2, 'k': 12, 'j': 25, 'm': 3, 'l': 6, 'o': 24, 'n': 14, 'q': 13, 'p': 10, 's': 15, 'r': 16, 'u': 1, 't': 8, 'w': 7, 'v': 4, 'y': 21, 'x': 22, 'z': 18}

In [ ]:
len(k)

26

In [10]:
################## TFIDF 

import pandas as pd
import numpy as np

df_train = pd.read_csv('xtrain_obfuscated.txt',header=None)
df_Y_train = pd.read_csv('ytrain.txt',header=None)

df_Y_train.rename(columns={0:'y_train'},inplace=True)
df_train.rename(columns={0:'x_train'},inplace=True)

train = pd.concat([df_train,df_Y_train],axis=1)

In [11]:
train.tail(5)

,x_train,y_train
32508,gzpmamqvmvdfuhtwamuluhvitwamdfuhvipmonmvlepmam...,6
32509,qvkrpmmvyppmletwxeuhiwenletwmktwqvuhvidfpmskuh...,10
32510,qvmvamuhleentwtvuhpmuluhskvienuhqgtwxemviwuhqv...,10
32511,ralrvimvamypenpmlegzgzenuhqvendfuhgztwulqvuhtw...,6
32512,twiwtvenvipmmvulqvtwkrszuhqvmvamuhsktwmktwmvam...,3


In [43]:
corpus_lm = []
for i in range(0,len(train)):
    k = train['x_train'][i]
    #print(i)
    d1 = list(k)
    listToStr1 = ' '.join([str(elem) for elem in d1])
    corpus_lm.append(listToStr1)
           

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(token_pattern='(?u)\\b\\w+\\b',ngram_range=(1,4))

## The problem is not the stopwords, there are no stopwords by default. 
## Problem is that the sentences in your test case is too short (1 character).
cv.fit(corpus_lm)
print("Vocabulary: ", cv.vocabulary_)
# Encode the Document
vector = cv.transform(corpus_lm)
  
# Summarizing the Encoded Texts
print("Encoded Document is:")
print(vector.toarray())
y = train['y_train']

Vocabulary:  {'s': 11853, 'a': 0, 't': 12350, 'w': 14452, 'm': 7537, 'u': 12873, 'l': 6833, 'h': 3659, 'q': 10458, 'g': 2741, 'r': 11003, 'v': 13334, 'e': 1554, 'z': 16077, 'k': 5982, 'p': 9821, 'c': 1071, 'i': 4610, 'n': 8506, 'd': 1431, 'f': 2383, 'x': 15376, 'y': 15675, 's a': 11854, 'a t': 559, 't w': 12773, 'w a': 14453, 'a m': 288, 'm u': 8211, 'u l': 13101, 'l u': 7400, 'u h': 12954, 'h q': 4165, 'q g': 10509, 'g u': 3335, 'l a': 6834, 'm l': 7901, 'l r': 7273, 'r m': 11404, 'm v': 8279, 'v e': 13431, 'e z': 2311, 'z u': 16578, 'q v': 10880, 'v k': 13722, 'k r': 6458, 'r p': 11470, 'p m': 10076, 'l e': 6877, 'e t': 2103, 'w u': 15162, 'l c': 6861, 'c i': 1156, 'i t': 5113, 'w s': 15055, 's k': 11961, 'k u': 6648, 'h l': 3957, 'e m': 1844, 'v t': 14057, 'h i': 3827, 'i w': 5236, 'w i': 14671, 'w e': 14560, 'e n': 1897, 'n u': 9087, 'r v': 11691, 'v i': 13591, 'i m': 4925, 'v q': 13945, 'r u': 11639, 'h u': 4395, 'n a': 8507, 'g q': 3133, 'w v': 15234, 'h t': 4336, 'l q': 7242, 'r

In [61]:
len(cv.vocabulary_)

16739

In [46]:
corpus_lm[0]

's a t w a m u l u h q g u l a m l r m v e z u h q v k r p m l e t w u l c i t w s k u h l e m v t w a m u l u h i w i w e n u h l r v i m v q v k r u h u l e n a m u l u h q g q v t w v i m v i w u h t w a m u l u h u l q v k r e n a m c i t w u h v i p m p m q v u h s k i w k r p m d f u h l r v i m v s k v i k r p m q v u h s k m v g z e n l e u h q v m v a m u l u h u l e n a m u l u h q v l e t w t w v i p m p m g z l e e n a m u h t w a m u l u h t w l e t w d f u h i w k r x e l e e n t w x e u h p m q v u h t w i w m v a m d f u h p k e z t w a m u l u h v i m v u h q v t w m k p m p m l e l r u h g z t w t w s k u h t w l r k r p m l r u h p m u l u h q v e n u h t w y p l e p m x e u h e n u h a m y p k r q v u h a m u l m v d f u h q v s k e n t w a m l e t w l r l r p m i w u h t w a m u l'

In [51]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vector, y, test_size=0.3, random_state=42)


from sklearn.naive_bayes import MultinomialNB
mb = MultinomialNB()
mb.fit(X_train, y_train)

y_pred=mb.predict(X_test)




from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("MNV accuracy: " ,accuracy)

print("TRAIN DATA accuracy:", mb.score(X_train,y_train))

print("TEST data accuracy: ", mb.score(X_test,y_test))

MNV accuracy:  0.7226778757432848
TRAIN DATA accuracy: 0.7480557142229448
TEST data accuracy:  0.7226778757432848


In [54]:
y_pred,len(y_pred),len(y_test)

(array([ 3, 10, 10, ...,  4,  1,  7]), 9754, 9754)

In [48]:

from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier()

clf_rf.fit(X_train,y_train)
rf_y_pred = clf_rf.predict(X_test)

print("Accuracy of train data: " , clf_rf.score(X_train,y_train))
print("Accuracy of test data: " , clf_rf.score(X_test,rf_y_pred))

from sklearn.metrics import accuracy_score
accuracy1 = accuracy_score(y_test, rf_y_pred)
print("Accuracy of RFC is:" , accuracy1)

Accuracy of train data:  1.0
Accuracy of test data:  1.0
Accuracy of RFC is: 0.664957965962682


In [50]:
#### overfitting issue in case of RandomForestClassifier 

In [55]:
alpha = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
accc = []
for i in alpha:
  mb = MultinomialNB(alpha=i)
  mb.fit(X_train, y_train)
  y_pred_cv = mb.predict(X_test)
  print("TRAIN DATA accuracy:", mb.score(X_train,y_train))
  print("TEST data accuracy: ", mb.score(X_test,y_test))
  accuracy2 = accuracy_score(y_test, y_pred_cv)
  accc.append(accuracy2)
  print("Count Vectoriser MNV classifier accuracy for value of alpha {} is: {}".format(i,accuracy2))




/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


TRAIN DATA accuracy: 0.8116788962608199
TEST data accuracy:  0.7408242772196022
Count Vectoriser MNV classifier accuracy for value of alpha 0 is: 0.7408242772196022
TRAIN DATA accuracy: 0.7647963443033525
TEST data accuracy:  0.730572072995694
Count Vectoriser MNV classifier accuracy for value of alpha 0.1 is: 0.730572072995694
TRAIN DATA accuracy: 0.7605782327870293
TEST data accuracy:  0.7283165880664343
Count Vectoriser MNV classifier accuracy for value of alpha 0.2 is: 0.7283165880664343
TRAIN DATA accuracy: 0.758381299705611
TEST data accuracy:  0.7272913676440435
Count Vectoriser MNV classifier accuracy for value of alpha 0.3 is: 0.7272913676440435
TRAIN DATA accuracy: 0.7561404279625643
TEST data accuracy:  0.7258560590526963
Count Vectoriser MNV classifier accuracy for value of alpha 0.4 is: 0.7258560590526963
TRAIN DATA accuracy: 0.7542510655125445
TEST data accuracy:  0.7251384047570227
Count Vectoriser MNV classifier accuracy for value of alpha 0.5 is: 0.7251384047570227
TRA

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range= (1,5),max_features= 100,token_pattern='(?u)\\b\\w+\\b')
x_new = tf.fit_transform(corpus_lm).toarray()

In [70]:
len(tf.vocabulary_)

100

In [71]:
len(tf.get_feature_names() )

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


100

In [72]:
X_final = pd.DataFrame(x_new, columns= tf.get_feature_names())
y = train['y_train']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42)

from sklearn.naive_bayes import MultinomialNB
mb = MultinomialNB()
mb.fit(X_train, y_train)

y_pred_tfid = mb.predict(X_test)

print("TRAIN DATA accuracy:", mb.score(X_train,y_train))

print("TEST data accuracy: ", mb.score(X_test,y_test))


from sklearn.metrics import accuracy_score
accuracy2 = accuracy_score(y_test, y_pred_tfid)
print("TFIDF MNV classifier accuracy: " ,accuracy2)

TRAIN DATA accuracy: 0.15870644580166088
TEST data accuracy:  0.15234775476727497
TFIDF MNV classifier accuracy:  0.15234775476727497


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [74]:
alpha = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
accc = []
for i in alpha:
  mb = MultinomialNB(alpha=i)
  mb.fit(X_train, y_train)
  y_pred_cv = mb.predict(X_test)
  print("TRAIN DATA accuracy:", mb.score(X_train,y_train))
  print("TEST data accuracy: ", mb.score(X_test,y_test))
  accuracy2 = accuracy_score(y_test, y_pred_cv)
  accc.append(accuracy2)
  print("Count Vectoriser MNV classifier accuracy for value of alpha {} is: {}".format(i,accuracy2))




/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


TRAIN DATA accuracy: 0.15870644580166088
TEST data accuracy:  0.15234775476727497
Count Vectoriser MNV classifier accuracy for value of alpha 0 is: 0.15234775476727497
TRAIN DATA accuracy: 0.15870644580166088
TEST data accuracy:  0.15234775476727497
Count Vectoriser MNV classifier accuracy for value of alpha 0.1 is: 0.15234775476727497
TRAIN DATA accuracy: 0.15870644580166088
TEST data accuracy:  0.15234775476727497
Count Vectoriser MNV classifier accuracy for value of alpha 0.2 is: 0.15234775476727497
TRAIN DATA accuracy: 0.15870644580166088
TEST data accuracy:  0.15234775476727497
Count Vectoriser MNV classifier accuracy for value of alpha 0.3 is: 0.15234775476727497
TRAIN DATA accuracy: 0.15870644580166088
TEST data accuracy:  0.15234775476727497
Count Vectoriser MNV classifier accuracy for value of alpha 0.4 is: 0.15234775476727497
TRAIN DATA accuracy: 0.15870644580166088
TEST data accuracy:  0.15234775476727497
Count Vectoriser MNV classifier accuracy for value of alpha 0.5 is: 0.

In [78]:
#### as compared to TFIDF --- Count vectoriser works well in this use case along with Multi nomial naive bays

In [100]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(token_pattern='(?u)\\b\\w+\\b',ngram_range=(1,4),max_features=10000)

## The problem is not the stopwords, there are no stopwords by default. 
## Problem is that the sentences in your test case is too short (1 character).
cv.fit(corpus_lm)
print(len(cv.vocabulary_))
# Encode the Document
vector = cv.transform(corpus_lm)
  
# Summarizing the Encoded Texts
print("Encoded Document is:")
print(vector.toarray())
y = train['y_train']

10000
Encoded Document is:
[[17  0  0 ...  0  0  0]
 [12  0  0 ...  0  0  0]
 [13  0  0 ...  0  0  0]
 ...
 [14  0  0 ...  0  0  0]
 [19  0  0 ...  0  0  0]
 [18  0  0 ...  0  0  0]]


In [101]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vector, y, test_size=0.3, random_state=42)


from sklearn.naive_bayes import MultinomialNB
mb = MultinomialNB(alpha=0)
mb.fit(X_train, y_train)

y_pred=mb.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("MNV accuracy: " ,accuracy)

print("TRAIN DATA accuracy:", mb.score(X_train,y_train))

print("TEST data accuracy: ", mb.score(X_test,y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


MNV accuracy:  0.7382612261636252
TRAIN DATA accuracy: 0.7999033349444176
TEST data accuracy:  0.7382612261636252


In [102]:
####### test data evaluation ####################

df_test = pd.read_csv('xtest_obfuscated.txt',header=None)

corpus_Test_data = []
for i in range(0,len(df_test)):
    k = df_test[0][i]
    #print(i)
    d1 = list(k)
    listToStr1 = ' '.join([str(elem) for elem in d1])
    corpus_Test_data.append(listToStr1)
           

In [87]:
len(corpus_Test_data)

3000

In [103]:
from sklearn.feature_extraction.text import CountVectorizer
cv2 = CountVectorizer(token_pattern='(?u)\\b\\w+\\b',ngram_range=(1,4),max_features=10000)

## The problem is not the stopwords, there are no stopwords by default. 
## Problem is that the sentences in your test case is too short (1 character).
cv2.fit(corpus_Test_data)
print(len(cv2.vocabulary_))
# Encode the Document
vector_test = cv2.transform(corpus_Test_data)
  
# Summarizing the Encoded Texts
print("Encoded Document is:")
print(vector_test.toarray())

10000
Encoded Document is:
[[15  0  0 ...  0  0  0]
 [18  0  0 ...  0  0  0]
 [14  0  0 ...  0  0  0]
 ...
 [20  1  1 ...  0  0  0]
 [15  0  0 ...  0  0  0]
 [18  0  0 ...  0  0  0]]


In [104]:
Y_test_data = mb.predict(vector_test)

In [105]:
Y_test_data

array([7, 6, 7, ..., 6, 7, 7])

In [111]:
with open("ytest.txt", "w") as output:
    for row in Y_test_data:
        output.write(str(row) + '\n')

In [112]:
df1 = pd.DataFrame(Y_test_data)

In [116]:
df1[0].value_counts() ################## output file 

7    2606
6     394
Name: 0, dtype: int64

In [117]:
df2_ypred = pd.DataFrame(y_pred)
df2_ypred[0].value_counts()

7     1909
6     1561
3     1062
1     1013
8      847
10     813
5      704
4      570
11     540
2      387
9      223
0      125
Name: 0, dtype: int64

In [ ]:
### Accuracy of 73.8 % using Countvectorizer with vocab size of 10000 and MULTINOMIAL NB algorithm.